Question 2 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
import random
import scipy.stats as stats
from scipy.stats import norm
import math


data=pd.read_csv('kernel_regression_1.csv')
x_ = np.linspace(-5,5,200)

size=len(data)
X=data.x.values
Y=data.y.values
k1 = lambda x,xp: math.exp(-3*abs(x-xp))
k2 = lambda x,xp: math.exp(-2*math.sqrt((x-xp)**2))
k3 = lambda x,xp: abs(x-xp)<0.5
kernel1 = lambda x: sum(k1(x,X[i])*Y[i] for i in range(size))/sum(k1(x,X[i]) for i in range(size))
kernel2 = lambda x: sum(k2(x,X[i])*Y[i] for i in range(size))/sum(k2(x,X[i]) for i in range(size))
kernel3= lambda x: sum(k3(x,X[i])*Y[i] for i in range(size))/sum(k3(x,X[i]) for i in range(size))

plt.figure(figsize=(20,8))
plt.scatter(data.x,data.y,color='pink',label='Raw Data')
plt.plot(x_,[kernel1(x) for x in x_],'red',label = 'Exponential Kernel')
plt.plot(x_,[kernel2(x) for x in x_],'yellow',label = 'Radial Basis Kernel')
plt.plot(x_,[kernel3(x) for x in x_],'blue',label = 'Uniform Kernel')
plt.legend(loc='upper right')
plt.title('Different kernels')

In [ ]:
data=pd.read_csv('kernel_regression_2.csv')
X=data.x.values
Y=data.y.values
Z=data.z.values
size=len(data)

k1 = lambda x,y,xp,yp: math.exp(-3*max(abs(x-xp),abs(y-yp)))
k2 = lambda x,y,xp,yp: math.exp(-2*math.sqrt((x-xp)**2+(y-yp)**2))
k3 = lambda x,y,xp,yp: max(abs(x-xp),abs(y-yp))<0.5

mhx1 = lambda x,y: sum(k1(x,y,X[i],Y[i])*Z[i] for i in range(size))/sum(k1(x,y,X[i],Y[i]) for i in range(size))
mhx2 = lambda x,y: sum(k2(x,y,X[i],Y[i])*Z[i] for i in range(size))/sum(k2(x,y,X[i],Y[i]) for i in range(size))
mhx3 = lambda x,y: sum(k3(x,y,X[i],Y[i])*Z[i] for i in range(size))/sum(k3(x,y,X[i],Y[i]) for i in range(size))

x_ = list(np.random.random(100)*5)+list(np.random.random(100)*(-5))
y_ = list(np.random.random(100)*5)+list(np.random.random(100)*(-5))
plt.figure(figsize=(15,8))
ax = plt.subplot(111,projection='3d')
ax.scatter(X,Y,Z,c='pink',label='RawData')
ax.plot_trisurf(x_,y_,[mhx1(x_[i],y_[i]) for i in range(200)],linewidth=0.1,label='Exponential Kernel')
plt.title('Exponential Kernel')

plt.figure(figsize=(15,8))
ax = plt.subplot(111,projection='3d')
ax.scatter(X,Y,Z,c='brown',label='RawData')
ax.plot_trisurf(x_,y_,[mhx2(x_[i],y_[i]) for i in range(200)],linewidth=0.1,label='Radial basis')
plt.title('Radial Basis Kernel')

plt.figure(figsize=(15,8))
ax = plt.subplot(111,projection='3d')
ax.scatter(X,Y,Z,c='green',label='RawData')
ax.plot_trisurf(x_,y_,[mhx3(x_[i],y_[i]) for i in range(200)],linewidth=0.1,label='Uniform')
plt.title('Uniform Kernel')


for bandwith in range(3,9):
    k2 = lambda x,y,xp,yp: math.exp(-2*math.sqrt(((x-xp)/bandwith)**2+((y-yp)/bandwith)**2))
    mhx2 = lambda x,y: sum(k2(x,y,X[i],Y[i])*Z[i] for i in range(size))/sum(k2(x,y,X[i],Y[i]) for i in range(size))
    plt.figure(figsize=(20,8))
    ax = plt.subplot(111,projection='3d')
    ax.scatter(X,Y,Z,c='black',label='RawData')
    ax.plot_trisurf(x_,y_,[mhx2(x_[i],y_[i]) for i in range(200)],linewidth=0.1,label='Radial basis')
    plt.title('Radial Basis Kernel with bandwith ' +str(bandwith))

Question 3

In [ ]:
data=pd.read_csv('svm.csv')
X=data[['x','intercept']].values
Y=data['y'].values
label=data['class'].values

def compute_cost(W, X, Y):
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances > 0] = 1
    hinge_loss = (np.sum(distances) / N)
    return abs(hinge_loss)

def calculate_gradient(W,X,Y,ratio):
    idx=random.sample(range(0, len(X)),int(len(X)*ratio))
    dw=np.zeros(len(W))
    for i in idx :
        if (Y[i]*np.dot(X[i], W)) < 1:
            di= (W-2*X[i] * Y[i])
        else:
            di=W
        dw+=di
    return dw/int(len(X)*ratio)
def eta(i):
    return 1.0/(i*2)
def svm_train(X,Y,ratio):
    max_epoch=201
    w=np.array([-800,0])
    res=[]
    for epoch in range(1,max_epoch):
        w=w-eta(epoch)*calculate_gradient(w,X,Y,ratio)
        loss=compute_cost(w, X, Y)
        res.append([epoch,w,loss])
    return w,res

w,res=svm_train(X,Y,0.7)
w

import time
run_time=[]
for i in np.linspace(0.1,1.0,45):
    start = time.time()
    _,_=svm_train(X,Y,i)
    stop = time.time()
    run_time.append([int(i*len(X)), stop-start])
    
import matplotlib.pyplot as plt
%matplotlib inline
x_vals = np.linspace(-10,10,1000)
y_vals = w[1] + w[0] * x_vals
plt.plot(x_vals, y_vals, '--',color='black')
plt.scatter(X[:,0],Y,c=label)
plt.title('Decsison Boundary')
plt.show()

res=np.array(res,object)
plt.plot(res[:,0],res[:,2],color='black')
plt.title('loss over time')

run_time=np.array(run_time)
plt.plot(run_time[:,0],run_time[:,1],color='blue')
plt.title('Run time for different n')

res=np.array(res,object)
plt.plot(res[:,0],res[:,2],color='black')
plt.title('loss over time')

run_time=np.array(run_time)
plt.plot(run_time[:,0],run_time[:,1],color='blue')
plt.title('Run time for different n')



Question 7

In [ ]:
data=pd.read_csv('gibbs.csv')
n = len(data)
beta = 4
alpha = 30
lambda_ = np.random.gamma(sum(data.value)+alpha,beta/(n*beta+1),888)

def likelihood_func(lambda_):
    res=1
    for value in data.value:
        res*=stats.poisson.pmf(value,lambda_)
    return -math.log(res)

lambda_.sort()
plt.plot(lambda_,[likelihood_func(l) for l in lambda_],color='black')
plt.title('Likelihood with different lambda')

plt.hist(lambda_,bins = 100,color='black')
plt.title('Posterior Distribution')

plt.hist(np.random.poisson(np.mean(lambda_),1000),bins=100,color='pink')
plt.title('Posterior Predictive Distribution')



Question 8

In [ ]:

import scipy.stats as stats
from scipy.stats import norm
import pymc3 as pm
import theano
from theano import tensor as T
from pymc3.math import switch
import matplotlib.pyplot as plt
import plotnine as pn
from plotnine import *
import corner 

In [ ]:
import pandas as pd 
data=pd.read_csv('change_point1.csv')
plt.plot(data['x'],data['y'],color='blue')
plt.show()

count_data.plot(x = 'time', y ='observed_counts',  kind = 'bar', figsize=(15,5))

mean_of_counts = count_data.observed_counts.mean()
alpha = 1.0/mean_of_counts
l = len(count_data)
model = pm.Model()

with model:

    lambda_1 = pm.Exponential('lambda_1',alpha)
    lambda_2 = pm.Exponential('lambda_2',alpha)

    tau = pm.DiscreteUniform('tau',lower = 0, upper= l-1)
    
with model:
    idx = count_data.time
    lambda_ = pm.math.switch(tau > idx, lambda_1, lambda_2) 
with model:
    observation = pm.Poisson("obs", lambda_, observed=count_data.observed_counts)
chains = 2
samples = 1000

with model:
    step = pm.Metropolis()
    trace = pm.sample(draws = samples, 
                      chains = chains, 
                      tune=5000,
                      step=step)

# - Posterior values -
lambda_1_samples =  trace['lambda_1']
lambda_2_samples = trace['lambda_2']
tau_samples = trace['tau']

# - Posterior distribution -
posterior_dist = pd.DataFrame({'lambda_1': lambda_1_samples, 'lambda_2': lambda_2_samples, 'tau' : tau_samples})
posterior_dist = posterior_dist.unstack().reset_index()
posterior_dist = posterior_dist.rename(columns = {'level_0':'Parameter',
                                                  'level_1':'Sample',
                                                  0:'simulated_parameter'})

import math
data=pd.read_csv('change_point1.csv')
niter = 2000 #number of iterations for the MCMC algorithm
t = np.arange(0,len(data)) #array of observation positions ('time')
data=data['y'].values

with pm.Model() as model:
    # define exponential prior
    alpha=pm.Exponential('alpha',1/np.mean(data))
    beta=pm.Exponential('beta',1/np.mean(data))
    tau=pm.DiscreteUniform("tau",t.min(),t.max())
    lambda_=T.switch(tau>=t, alpha, beta)
    ## note that the factorial constant is ignored
    logp = -lambda_ + T.log(lambda_)*data
    
    def logp_func(data):
        return logp.sum()
    
    L_obs=pm.DensityDist('L_obs',logp_func, observed=data)
    #start MCMC algorithm
    start = pm.find_MAP()
    #iterate MCMC
    trace = pm.sample(niter, start=start, random_seed=123, progressbar=True)



Question 9

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data=pd.read_csv('radon.csv')

from pymc3 import get_data
srrs2 = pd.read_csv(get_data('srrs2.dat'))
srrs2.columns = srrs2.columns.map(str.strip)
srrs_mn = srrs2[srrs2.state=='MN'].copy()
srrs_mn['fips'] = srrs_mn.stfips*1000 + srrs_mn.cntyfips
cty = pd.read_csv(get_data('cty.dat'))
cty_mn = cty[cty.st=='MN'].copy()
cty_mn[ 'fips'] = 1000*cty_mn.stfips + cty_mn.ctfips

srrs_mn = srrs_mn.merge(cty_mn[['fips', 'Uppm']], on='fips')
srrs_mn = srrs_mn.drop_duplicates(subset='idnum')
u = np.log(srrs_mn.Uppm)
srrs_mn.county = srrs_mn.county.map(str.strip)
mn_counties = srrs_mn.county.unique()
counties = len(mn_counties)
county_lookup = dict(zip(mn_counties, range(len(mn_counties))))

county = srrs_mn['county_code'] = srrs_mn.county.replace(county_lookup).values
radon = srrs_mn.activity
srrs_mn['log_radon'] = log_radon = np.log(radon + 0.1).values
floor_measure = srrs_mn.floor.values

srrs_mn.activity.apply(lambda x: np.log(x+0.1)).hist(bins=25);

from pymc3 import Deterministic
from pymc3 import Model, sample, Normal, HalfCauchy, Uniform, model_to_graphviz
with Model() as hierarchical_intercept:
    # Priors
    sigma_a = HalfCauchy('sigma_a', 5)
    
    # County uranium model for slope
    gamma_0 = Normal('gamma_0', mu=0., sigma=1e5)
    gamma_1 = Normal('gamma_1', mu=0., sigma=1e5)
    # Uranium model for intercept
    mu_a = gamma_0 + gamma_1*u
    # County variation not explained by uranium
    eps_a = Normal('eps_a', mu=0, sigma=sigma_a, shape=counties)
    a = Deterministic('a', mu_a + eps_a[county])
    
    # Common slope
    b = Normal('b', mu=0., sigma=1e5)
    
    # Model error
    sigma_y = Uniform('sigma_y', lower=0, upper=100)
    
    # Expected value
    y_hat = a + b * floor_measure
    # Data likelihood
    y_like = Normal('y_like', mu=y_hat, sigma=sigma_y, observed=log_radon)
model_to_graphviz(hierarchical_intercept)

with hierarchical_intercept:
    hierarchical_intercept_trace = sample(1000,tune=1000)
    
import pymc3 as pm
pm.traceplot(hierarchical_intercept_trace,legend=True)

a_means = hierarchical_intercept_trace['a'].mean(axis=0)
b_means = hierarchical_intercept_trace['b'].mean(axis=0)
y_hat = a_means + b_means * floor_measure

data=srrs_mn[['county','log_radon','log_radon_predict']].groupby('county').mean().reset_index()

plt.figure(figsize=(25,8))
plt.bar(data['county'],data['log_radon'])
plt.xticks(rotation='vertical')
plt.title('Average_log_Randon_level_by_county')
plt.show()

plt.figure(figsize=(25,8))
plt.bar(data['county'],data['log_radon_predict'],color='orange')
plt.xticks(rotation='vertical')
plt.title('Predicted_Average_log_Randon_level_by_county')
plt.show()

